# vcc792_bin.ipynb

### Isaac Cheng - September 2021

Various plots of VCC 792 with binning. Read [this webpage](http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/community/ngvs/docs/ngvsdoc.html) for lots of useful information! Also look at [this PDF](https://www.ucolick.org/~bolte/AY257/s_n.pdf) for a useful overview of CCD signal/noise satistics. Also read the section on binning.

Also: [Astropy Course](https://www.iasf-milano.inaf.it/astropy-course/), in particular, see the [Astropy image walkthrough](https://www.iasf-milano.inaf.it/AstropyCourse/07.Images.slides.html).

[vorbin manual](https://www-astro.physics.ox.ac.uk/~mxc/software/vorbin_manual.pdf) and [Michele Cappellari's Python Programs page](http://www-astro.physics.ox.ac.uk/~mxc/software/)

### Current Goals

- Display voronoi binned data without using built-in function?
  - For major axis profile
  - To see pixelation
  - Likely will need to adapt `vorbin` to return more variables.
- Use RA & Dec to display coordinates of binned images


In [1]:
# # May need to do this if running remote server with VS Code
%cd "/arc/home/IsaacCheng/coop_f2021/warmup"

/arc/home/IsaacCheng/coop_f2021/warmup


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
from astropy.wcs import WCS
from astropy.nddata import block_reduce
import vorbin
from vorbin.voronoi_2d_binning import voronoi_2d_binning
import plotbin
import fits_plot_utils as utils
import reproject
from reproject import reproject_interp, reproject_exact
import dill
import pandas as pd
%matplotlib widget

# Voronoi Binning


In [ ]:
# 
# Load data
# 
vcc792_dist = 16.5 * u.Mpc
# Signal
iband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184.fits"
gband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits"
uband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184.fits"
# Noise (sigma maps)
inoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.sig.3136_8588_6905_10184.fits"
gnoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.sig.3136_8588_6905_10184.fits"
unoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.sig.3136_8588_6905_10184.fits"
# Mask (flap maps)
igood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.flag.3136_8588_6905_10184.fits"
ggood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.flag.3136_8588_6905_10184.fits"
ugood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.flag.3136_8588_6905_10184.fits"
# 
# Load NGVS data
# 
# Signal
data_i, header_i = utils.load_img(iband_path)
data_g, header_g = utils.load_img(gband_path)
data_u, header_u = utils.load_img(uband_path)
# Noise
noise_i, header_noise_i = utils.load_img(inoise_path)
noise_g, header_noise_g = utils.load_img(gnoise_path)
noise_u, header_noise_u = utils.load_img(unoise_path)
# Masks
good_i, header_good_i = utils.load_img(igood_path, idx=1)
good_g, header_good_g = utils.load_img(ggood_path, idx=1)
good_u, header_good_u = utils.load_img(ugood_path, idx=1)
# 
# Cut out galaxy
# 
vcc792_center = (2716, 1613)
vcc792_shape = (1480, 1127)
data_i_cut, wcs_i_cut = utils.cutout(data_i, vcc792_center, vcc792_shape, header=header_i)
data_g_cut, wcs_g_cut = utils.cutout(data_g, vcc792_center, vcc792_shape, header=header_g)
data_u_cut, wcs_u_cut = utils.cutout(data_u, vcc792_center, vcc792_shape, header=header_u)
noise_i_cut, wcs_noise_i_cut = utils.cutout(noise_i, vcc792_center, vcc792_shape, header=header_noise_i)
noise_g_cut, wcs_noise_g_cut = utils.cutout(noise_g, vcc792_center, vcc792_shape, header=header_noise_g)
noise_u_cut, wcs_noise_u_cut = utils.cutout(noise_u, vcc792_center, vcc792_shape, header=header_noise_u)
# 
# VERTICO CO Data
# 
# co_path = "/arc/projects/vertico/products/release.v1.2/native/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_mom0_Msun.fits"
co_path = "/arc/projects/vertico/products/release.v1.2/nyquistPix/9arcsec/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_nyquistPix_9_arcsec_mom0_Msun.fits"
data_co, header_co = utils.load_img(co_path)

In [5]:
def calc_colour_err(blue, red, blue_err, red_err):
    """
    Calculates the uncertainty in the colour index using basic uncertainty propagation.
    The colour, or colour index, is defined as:
                        colour = -2.5 * log10(blue / red)
    where blue is the flux in the shorter wavelength band and red is the flux in the
    longer wavelength band.
    The uncertainty, assuming independent errors and to a first-order approximation, is
    given by:
            colour_err^2 = (-2.5/ln(10) * 1/blue)^2 * blue_err^2 +
                           (+2.5/ln(10) * 1/red)^2 * red_err^2
                         = (2.5/ln(10))^2 * [(blue_err/blue)^2 + (red_err/red)^2]
    Thus:
            colour_err = sqrt(colour_err^2)
                       = 2.5 / ln(10) * sqrt((blue_err/blue)^2 + (red_err/red)^2)
    Note that all the parameters MUST be able to broadcast together.

    Parameters:
      blue :: array
        The flux in the shorter wavelength band
      red :: array
        The flux in the longer wavelength band
      blue_err :: array
        The uncertainty in the flux in the shorter wavelength band
      red_err :: array
        The uncertainty in the flux in the longer wavelength band
        
    Returns: colour_err
      colour_err :: array
        The uncertainty in the colour index.
    """
    prefactor = 2.5 / np.log(10)
    errs = np.sqrt((blue_err / blue) ** 2 + (red_err / red) ** 2)
    return prefactor * errs


def prelim_bin(signal, noise, block_size=(4, 4), print_info=True, func=np.nansum):
    """
    Regular preliminary binning of 2D data (e.g., for Voronoi binning). Ignores NaNs.
    
    (From astropy's block_reduce() documentation): If the data are not perfectly divisible
    by block_size along a given axis, then the data will be trimmed (from the end) along
    that axis.

    TODO: finish docstring
    """
    # Add signal array using arithmetic sum
    signal_binned = block_reduce(signal, block_size=block_size, func=func)
    # Add noise array in quadrature
    noise_binned = noise * noise
    noise_binned = block_reduce(noise_binned, block_size=block_size, func=func)
    noise_binned = np.sqrt(noise_binned)
    # 
    # Generate pixel coordinates
    # 
    y_coords, x_coords = np.meshgrid(
        np.arange(signal_binned.shape[0]), np.arange(signal_binned.shape[1])
    )
    x_coords, y_coords = x_coords.T, y_coords.T
    # 
    # Ensure no infs or NaNs in binned data (for Voronoi binning)
    # 
    is_good = np.isfinite(signal_binned) & np.isfinite(noise_binned)
    # 
    if print_info:
        print("x_coords, y_coords, signal_binned, noise_binned shapes:",
              x_coords.shape, y_coords.shape, signal_binned.shape, noise_binned.shape)
        print("total bad elements (infs/NaNs):", np.sum(~is_good))
    # 
    return signal_binned, noise_binned, x_coords, y_coords, is_good


def optimal_sn(index, signal, noise):
    """
    Signal-to-noise ratio approximation using optimal weighing of pixels.

    See Eq. (3) of Cappellari & Copin (2003):
    https://ui.adsabs.harvard.edu/abs/2003MNRAS.342..345C/abstract
    
    Parameters: (nearly verbatim from Cappellari & Copin's voronoi_2d_binning.py)
      index :: 1D array
        Integer vector of length N containing the indices of
        the spaxels for which the combined S/N has to be returned.
        The indices refer to elements of the vectors signal and noise.
      signal :: 1D array
        Vector of length M>N with the signal of all spaxels.
      noise :: 1D array
        Vector of length M>N with the noise of all spaxels.
    
    Returns: sn
      sn :: 1D array
        Scalar S/N or another quantity that needs to be equalized.
    """
    return np.sqrt(
        np.sum((signal[index] / noise[index]) * (signal[index] / noise[index]))
    )


## Debugging Stuff


In [ ]:
# 
# Explicitly setting array values to floats
# 
# Generate 2500 (total, not 2500 x 2500) uniform pixels
x_coords = np.arange(1,51).astype(int)
y_coords = np.arange(1,51).astype(int)
x_coords, y_coords = np.meshgrid(x_coords, y_coords)
gi_colour = np.full(2500, 5.0)
gi_colour_err = np.full(2500, 1.0)
# 
binNum, xNode, yNode, xBar, yBar, sn, nPixels, scale = voronoi_2d_binning(
    y_coords.flatten(),
    x_coords.flatten(),
    gi_colour.flatten(),
    gi_colour_err.flatten(),
    targetSN=10,
    pixelsize=None,
    cvt=False,
    quiet=True,
    wvt=False,
    sn_func=None,
)
cmap = "plasma"
cmap = mpl.cm.get_cmap(cmap)
#
fig, ax = plt.subplots()
img = ax.scatter(
    x_coords.flatten(), y_coords.flatten(), c=(gi_colour/gi_colour_err), cmap=cmap, s=1
)
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of Original Data (all 5.0)")
ax.set_title("Random data for debugging\\ldots")
# ax.grid(False)
ax.set_aspect("equal")
# fig.savefig("imgs/raw_random_data.pdf")
plt.show()

## g-i Colour


In [ ]:
%matplotlib inline
gi_colour = -2.5 * np.log10(data_g_cut / data_i_cut)
gi_colour_err = calc_colour_err(data_g_cut, data_i_cut, noise_g_cut, noise_i_cut)
# 
# Preliminary binning
#
block_size = (4, 4)
gi_colour = block_reduce(gi_colour, block_size=block_size)
gi_colour_err = block_reduce(gi_colour_err, block_size=block_size)
# 
# Generate pixel coordinates (needed for vorbin)
# 
y_coords, x_coords = np.meshgrid(
    np.arange(gi_colour.shape[0]), np.arange(gi_colour.shape[1])
)
x_coords, y_coords = x_coords.T, y_coords.T
is_good = np.isfinite(gi_colour_err) & np.isfinite(gi_colour)
# 
# Printing some info
# 
print("array shapes:",
      x_coords.shape, y_coords.shape, gi_colour.shape, gi_colour_err.shape)
print("sum bad:", np.sum(~is_good))
# 
# Plot data passed into voronoi_2d_binning()
# 
gi_snr = abs(np.ma.masked_where(~is_good, gi_colour / gi_colour_err))
cmap = "plasma"
cmap = mpl.cm.get_cmap(cmap)
#
# fig, ax = plt.subplots(subplot_kw={"projection": wcs_i_cut})  # auto-sets origin="lower"
fig, ax = plt.subplots()
# img = ax.scatter(
#     x_coords.flatten(), y_coords.flatten(), c=(gi_colour/gi_colour_err), cmap=cmap, s=1
# )
img = ax.imshow(gi_snr, cmap=cmap, origin="lower", vmax=90) 
# img = ax.imshow(gi_colour, cmap=cmap, vmin=0.5, vmax=1.6)
# img = ax.imshow(gi_colour_err, cmap=cmap, vmax=np.percentile(gi_colour_err[is_good], 15))
# utils.add_scalebar(ax, wcs_i_cut, vcc792_dist,
#                    scalebar_factor=2, label="2 kpc", color="w", loc="lower right")
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
# cbar.set_label("g-i Colour Index")
# cbar.set_label("SNR (g-i colour / error)")
cbar.set_label("SNR of Data Passed into \\texttt{vorbin}")
#
# Overplot CO contours from VERTICO data
#
# ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
# 
# ax.tick_params(which="both", direction="out")
# ax.set_xlabel("RA (J2000)")
# ax.set_ylabel("Dec (J2000)")
ax.set_title("VCC 792 / NGC 4380: g-i Colour")
ax.grid(False)
ax.set_aspect("equal")
# fig.savefig("imgs/vcc792_g-i_binned_4x4.pdf")
plt.show()
# 
# ! LARGE IMAGE! Voronoi binning takes a long time. See method docstring
#
gi_binNum, gi_xNode, gi_yNode, gi_xBar, gi_yBar, gi_sn, gi_nPixels, gi_scale = voronoi_2d_binning(
    x_coords[is_good],
    y_coords[is_good],
    gi_colour[is_good],
    gi_colour_err[is_good],
    targetSN=50,
    pixelsize=1,
    cvt=False,
    quiet=False,
    wvt=False,
    sn_func=optimal_sn,
)


In [ ]:
gi_binNum.max()

## u-band


In [ ]:
%matplotlib inline
# 
# Preliminary binning
#
# 2 x 2 binning: ~2120 seconds (> 35 min!), 4 x 4 binning: ~ 111 seconds
data_u_cut_binned, noise_u_cut_binned, x_coords_u, y_coords_u, is_good_u = prelim_bin(
    data_u_cut, noise_u_cut, block_size=(4, 4), print_info=True
)
# 
# Plot data passed into voronoi_2d_binning()
# 
u_snr = abs(np.ma.masked_where(~is_good_u, data_u_cut_binned / noise_u_cut_binned))
cmap = "plasma"
cmap = mpl.cm.get_cmap(cmap)
#
fig, ax = plt.subplots()
img = ax.imshow(u_snr, cmap=cmap, origin="lower", vmax=90) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of Data Passed into \\texttt{vorbin}")
ax.set_title("VCC 792 / NGC 4380: u-band Data")
ax.grid(False)
ax.set_aspect("equal")
fig.savefig("imgs/vcc792_u_binned_4x4.pdf")
plt.show()
# 
# Voronoi binning
#
u_binNum, u_xNode, u_yNode, u_xBar, u_yBar, u_sn, u_nPixels, u_scale = voronoi_2d_binning(
    x_coords_u[is_good_u],
    y_coords_u[is_good_u],
    data_u_cut_binned[is_good_u],
    noise_u_cut_binned[is_good_u],
    targetSN=50,
    pixelsize=1,
    cvt=False,
    quiet=False,
    wvt=False,
    sn_func=None,
)


In [ ]:
plt.close("all")

# Binning to VERTICO Nyquist Resolution


## Loading data


In [3]:
# 
# Load data
# 
vcc792_dist = 16.5 * u.Mpc
# Signal
zband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.3136_8588_6905_10184.fits"
iband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184.fits"
gband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits"
uband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184.fits"
# Noise (sigma maps)
znoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.sig.3136_8588_6905_10184.fits"
inoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.sig.3136_8588_6905_10184.fits"
gnoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.sig.3136_8588_6905_10184.fits"
unoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.sig.3136_8588_6905_10184.fits"
# Mask (flap maps)
zgood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.flag.3136_8588_6905_10184.fits"
igood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.flag.3136_8588_6905_10184.fits"
ggood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.flag.3136_8588_6905_10184.fits"
ugood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.flag.3136_8588_6905_10184.fits"
# 
# Load NGVS data
# 
# Signal
data_z, header_z = utils.load_img(zband_path)
data_i, header_i = utils.load_img(iband_path)
data_g, header_g = utils.load_img(gband_path)
data_u, header_u = utils.load_img(uband_path)
# Noise
noise_z, header_noise_z = utils.load_img(znoise_path)
noise_i, header_noise_i = utils.load_img(inoise_path)
noise_g, header_noise_g = utils.load_img(gnoise_path)
noise_u, header_noise_u = utils.load_img(unoise_path)
# Masks
good_z, header_good_z = utils.load_img(zgood_path, idx=1)
good_i, header_good_i = utils.load_img(igood_path, idx=1)
good_g, header_good_g = utils.load_img(ggood_path, idx=1)
good_u, header_good_u = utils.load_img(ugood_path, idx=1)
# 
# VERTICO CO Data
# 
# co_path = "/arc/projects/vertico/products/release.v1.2/native/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_mom0_Msun.fits"
co_path = "/arc/projects/vertico/products/release.v1.2/nyquistPix/9arcsec/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_nyquistPix_9_arcsec_mom0_Msun.fits"
data_co, header_co = utils.load_img(co_path)
wcs_co = WCS(header_co)

Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      54   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      52   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      52   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      58   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.sig.3136_8588_6905_10

## Useful functions for binning

In [19]:
def get_reproject_shape_factor(target_arr, input_wcs, target_wcs):
    #
    # Determine reprojection shape and binning factor
    #   N.B. reprojection shape should be as close as possible to the shape of a regular
    #   "cut out" if the input array was cut at the boundaries of the target.
    #
    # Find the coordinates of the target_data's edges, assuming the data are rectangular
    target_bottomleft = target_wcs.pixel_to_world(0, 0)
    target_topright = target_wcs.pixel_to_world(*target_arr.shape)
    # ? Don't know if I am passing the arguments in the right order.
    target_centre = target_wcs.pixel_to_world(
        target_arr.shape[1] / 2, target_arr.shape[0] / 2
    )
    # Map the pixels above to their corresponding pixels in the input array
    input_bottomleft = input_wcs.world_to_pixel(target_bottomleft)
    input_topright = input_wcs.world_to_pixel(target_topright)
    input_centre = input_wcs.world_to_pixel(target_centre)
    # Determine binning/transformation factor
    input_to_target_factor = np.subtract(input_topright, input_bottomleft)
    input_to_target_factor = np.round(
        np.divide(input_to_target_factor, target_arr.shape)
    ).astype(int)
    input_reproject_shape = input_to_target_factor * target_arr.shape
    #
    return input_to_target_factor, input_reproject_shape


def reproj_arr(
    input_arr,
    input_wcs,
    target_wcs,
    input_to_target_factor,
    input_reproject_shape,
    reproject_func=reproject.reproject_exact,
):
    """
    Reproject input array to target array.
    
    target_wcs must support CDELT keyword.
    TODO: add support for CD[#]_[#] keywords.
    
    Requires the reproject package: https://reproject.readthedocs.io/en/stable/
    
    TODO: finish docstring
    """
    #
    # Reproject input array to target array
    #
    wcs_reproject = target_wcs.deepcopy()
    wcs_reproject.wcs.crpix = target_wcs.wcs.crpix * input_to_target_factor
    wcs_reproject.wcs.cdelt = target_wcs.wcs.cdelt / input_to_target_factor
    wcs_reproject.array_shape = input_reproject_shape
    arr_reproject = reproject_func(
        (input_arr, input_wcs),
        wcs_reproject,
        shape_out=input_reproject_shape,
        return_footprint=False,
    )
    return arr_reproject, wcs_reproject


def bin_snr_to_target(
    signal_arr,
    signal_wcs,
    noise_arr,
    noise_wcs,
    target_arr,
    target_wcs,
    reproject_func=reproject.reproject_exact,
    bin_func=np.nansum,
    print_debug=False,
):
    """
    Wrapper for binning an signal & noise arrays to the exact physical dimensions and
    resolution of a target (provided via the target_wcs). The input arrays should already
    be masked (i.e., invalid values should be set to np.nan) and the input arrays should
    entirely contain the target_wcs (i.e., the extent of the target_data).

    Requires the reproject package: https://reproject.readthedocs.io/en/stable/

    target_wcs should support the CDELT keyword.
    TODO: add support for CD[#]_[#] keywords.

    TODO: finish docstring
    """
    if print_debug:
        print("target_wcs:", target_wcs)
        print()
    #
    # Determine reprojection shape and binning factor
    #   N.B. reprojection shape should be as close as possible to the shape of a regular
    #   "cut out" if the input array was cut at the boundaries of the target.
    #
    signal_to_target_factor, signal_reproject_shape = get_reproject_shape_factor(
        target_arr, signal_wcs, target_wcs
    )
    if print_debug:
        print("signal_to_target_factor:", signal_to_target_factor)
        print("signal_reproject_shape:", signal_reproject_shape)
        print()
    noise_to_target_factor, noise_reproject_shape = get_reproject_shape_factor(
        target_arr, noise_wcs, target_wcs
    )
    if np.any(signal_reproject_shape != noise_reproject_shape) or np.any(
        signal_to_target_factor != noise_to_target_factor
    ):
        raise ValueError("Signal and noise arrays must have the same shape and wcs.")
    #
    # Reproject data
    #
    # N.B. signal_wcs_reproject and noise_wcs_reproject should be the same
    signal_reproject, signal_wcs_reproject = reproj_arr(
        signal_arr,
        signal_wcs,
        target_wcs,
        signal_to_target_factor,
        signal_reproject_shape,
        reproject_func=reproject_func,
    )
    if print_debug:
        print("signal_wcs_reproject:", signal_wcs_reproject)
        print()
    noise_reproject, noise_wcs_reproject = reproj_arr(
        noise_arr,
        noise_wcs,
        target_wcs,
        noise_to_target_factor,
        noise_reproject_shape,
        reproject_func=reproject_func,
    )
    #
    # Bin to target resolution
    #
    signal_binned, noise_binned, x_coords, y_coords, is_good = prelim_bin(
        signal_reproject,
        noise_reproject,
        block_size=signal_to_target_factor,
        func=bin_func,
        print_info=print_debug,
    )
    # Modify WCS object
    # ? Not sure if slice argument order is correct
    wcs_binned = signal_wcs_reproject.slice(
        (np.s_[:: signal_to_target_factor[0]], np.s_[:: signal_to_target_factor[1]])
    )
    wcs_binned.wcs.crpix = signal_wcs_reproject.wcs.crpix / signal_to_target_factor
    wcs_binned.wcs.cdelt = signal_wcs_reproject.wcs.cdelt * signal_to_target_factor
    if print_debug:
        print("wcs_binned:", wcs_binned)
    #
    return x_coords, y_coords, signal_binned, noise_binned, is_good, wcs_binned


def dill_data(
    outfile,
    reproject_method,
    x_coords,
    y_coords,
    signal_binned,
    noise_binned,
    is_good,
    wcs_binned,
    wcs_binned_array_shape,
):
    with open(outfile, "wb") as f:
        dill.dump(
            {
                "note": "Remember to set `wcs_binned.array_shape = wcs_binned_array_shape`",
                "reproject_method": reproject_method,
                "x_coords": x_coords,
                "y_coords": y_coords,
                "signal_binned": signal_binned,
                "noise_binned": noise_binned,
                "is_good": is_good,
                "wcs_binned": wcs_binned,
                "wcs_binned_array_shape": wcs_binned_array_shape,  # this is the "NAXIS" keyword
            },
            f,
        )
    print(f"Pickled {outfile}")


def txt_data(
    outfile,
    xs,
    ys, 
    u_sig,
    u_noise,
    g_sig,
    g_noise,
    i_sig,
    i_noise,
    z_sig,
    z_noise,
):
    id_arr = np.array(
        [str(x).zfill(2) + str(y).zfill(2) for x, y in zip(xs.flatten(), ys.flatten())]
    )
    df = pd.DataFrame(
        {
            "id": id_arr,
            "z": np.zeros(xs.size),
            "counts_u": u_sig.flatten(),
            "err_u": u_noise.flatten(),
            "counts_g": g_sig.flatten(),
            "err_g": g_noise.flatten(),
            "counts_i": i_sig.flatten(),
            "err_i": i_noise.flatten(),
            "counts_z": z_sig.flatten(),
            "err_z": z_noise.flatten(),
        }
    )
    df.to_csv(path_or_buf=outfile, sep=" ", index=False, header=True)
    print(f"Saved {outfile}")


def plot_snr(
    signal,
    noise,
    wcs,
    contour_arr,
    countour_wcs,
    contour_levels=[0, 5, 10],
    vmin=150,
    vmax=1250,
    band="u-band",
):
    snr = abs(signal / noise)
    fig, ax = plt.subplots(subplot_kw={"projection": wcs})
    img = ax.imshow(snr, cmap="plasma", vmin=vmin, vmax=vmax)
    cbar = fig.colorbar(img)
    cbar.ax.tick_params(which="both", direction="out")
    cbar.set_label(f"{band} SNR")
    ax.contour(
        contour_arr,
        transform=ax.get_transform(countour_wcs),
        levels=contour_levels,
        colors="w",
    )
    ax.set_title(f"VCC 792 / NGC 4380: {band} Data Binned")
    ax.set_xlabel("RA (J2000)")
    ax.set_ylabel("Dec (J2000)")
    ax.grid(False)
    ax.set_aspect("equal")
    plt.show()

## Process & bin data


In [20]:
#
# Mask the data and noise arrays
#
data_u_masked = np.ma.masked_array(data_u, mask=good_u == 0).filled(fill_value=np.nan)
noise_u_masked = np.ma.masked_array(noise_u, mask=good_u == 0).filled(fill_value=np.nan)
data_g_masked = np.ma.masked_array(data_g, mask=good_g == 0).filled(fill_value=np.nan)
noise_g_masked = np.ma.masked_array(noise_g, mask=good_g == 0).filled(fill_value=np.nan)
data_i_masked = np.ma.masked_array(data_i, mask=good_i == 0).filled(fill_value=np.nan)
noise_i_masked = np.ma.masked_array(noise_i, mask=good_i == 0).filled(fill_value=np.nan)
data_z_masked = np.ma.masked_array(data_z, mask=good_z == 0).filled(fill_value=np.nan)
noise_z_masked = np.ma.masked_array(noise_z, mask=good_z == 0).filled(fill_value=np.nan)

In [21]:
reproj_func_used = reproject.reproject_interp
reproj_method_used = "reproject_interp"

### u-band


In [22]:
plt.close("all")
# Bin NGVS data to VERTICO resolution
(
    u_xs_bin,
    u_ys_bin,
    u_signal_bin,
    u_noise_bin,
    u_isgood_bin,
    u_wcs_bin,
) = bin_snr_to_target(
    data_u_masked,
    WCS(header_u),
    noise_u_masked,
    WCS(header_noise_u),
    data_co,
    wcs_co,
    reproject_func=reproj_func_used,
    bin_func=np.nansum,
    print_debug=False,
)
plot_snr(u_signal_bin, u_noise_bin, u_wcs_bin, data_co, wcs_co, vmin=150, vmax=1250, band="u-band")
# Pickle data
u_outfile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.u.Mg004.sig.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
dill_data(u_outfile, reproj_method_used, u_xs_bin, u_ys_bin, u_signal_bin, u_noise_bin, u_isgood_bin, u_wcs_bin, u_wcs_bin.array_shape)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pickled /arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.u.Mg004.sig.3136_8588_6905_10184_BINNED_reproject_interp.pkl


### g-band


In [23]:
plt.close("all")
# Bin NGVS data to VERTICO resolution
(
    g_xs_bin,
    g_ys_bin,
    g_signal_bin,
    g_noise_bin,
    g_isgood_bin,
    g_wcs_bin,
) = bin_snr_to_target(
    data_g_masked,
    WCS(header_g),
    noise_g_masked,
    WCS(header_noise_g),
    data_co,
    wcs_co,
    reproject_func=reproj_func_used,
    bin_func=np.nansum,
    print_debug=False,
)
plot_snr(g_signal_bin, g_noise_bin, g_wcs_bin, data_co, wcs_co, vmin=400, vmax=3000, band="g-band")
# Pickle data
g_outfile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.g.Mg004.sig.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
dill_data(g_outfile, reproj_method_used, g_xs_bin, g_ys_bin, g_signal_bin, g_noise_bin, g_isgood_bin, g_wcs_bin, g_wcs_bin.array_shape)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pickled /arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.g.Mg004.sig.3136_8588_6905_10184_BINNED_reproject_interp.pkl


### i-band


In [24]:
plt.close("all")
# Bin NGVS data to VERTICO resolution
(
    i_xs_bin,
    i_ys_bin,
    i_signal_bin,
    i_noise_bin,
    i_isgood_bin,
    i_wcs_bin,
) = bin_snr_to_target(
    data_i_masked,
    WCS(header_i),
    noise_i_masked,
    WCS(header_noise_i),
    data_co,
    wcs_co,
    reproject_func=reproj_func_used,
    bin_func=np.nansum,
    print_debug=False,
)
plot_snr(i_signal_bin, i_noise_bin, i_wcs_bin, data_co, wcs_co, vmin=300, vmax=3000, band="i-band")
# Pickle data
i_outfile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.i.Mg004.sig.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
dill_data(i_outfile, reproj_method_used, i_xs_bin, i_ys_bin, i_signal_bin, i_noise_bin, i_isgood_bin, i_wcs_bin, i_wcs_bin.array_shape)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pickled /arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.i.Mg004.sig.3136_8588_6905_10184_BINNED_reproject_interp.pkl


In [25]:
plt.close("all")
# Bin NGVS data to VERTICO resolution
# z-band
(
    z_xs_bin,
    z_ys_bin,
    z_signal_bin,
    z_noise_bin,
    z_isgood_bin,
    z_wcs_bin,
) = bin_snr_to_target(
    data_z_masked,
    WCS(header_z),
    noise_z_masked,
    WCS(header_noise_z),
    data_co,
    wcs_co,
    reproject_func=reproj_func_used,
    bin_func=np.nansum,
    print_debug=False,
)
plot_snr(z_signal_bin, z_noise_bin, z_wcs_bin, data_co, wcs_co, vmin=200, vmax=2000, band="z-band")
# Pickle data
z_outfile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.z.Mg004.sig.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
dill_data(z_outfile, reproj_method_used, z_xs_bin, z_ys_bin, z_signal_bin, z_noise_bin, z_isgood_bin, z_wcs_bin, z_wcs_bin.array_shape)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pickled /arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.z.Mg004.sig.3136_8588_6905_10184_BINNED_reproject_interp.pkl


In [26]:
# 
# Read pickle files
# 
# with open(u_outfile, "rb") as fu:
    # file = dill.load(fu)
    # print(file.keys())
    # print(file["note"])
    # print(file["reproject_method"])
    # print(file["x_coords"])
    # print(file["y_coords"])
    # print(file["signal_binned"])
    # print(file["noise_binned"])
    # print(file["is_good"])
    # print(file["wcs_binned"])
    # print(file["wcs_binned_array_shape"])
# 
# Generate ASCII txt file
# 
txt_outfile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/vcc792_BINNED_{reproj_method_used}.txt"
txt_data(
    txt_outfile,
    u_xs_bin,
    u_ys_bin,
    u_signal_bin,
    u_noise_bin,
    g_signal_bin,
    g_noise_bin,
    i_signal_bin,
    i_noise_bin,
    z_signal_bin,
    z_noise_bin,
)

Saved /arc/home/IsaacCheng/coop_f2021/binned_data/vcc792_BINNED_reproject_interp.txt


## Old code (for testing, warmup, etc.)


In [ ]:
# 
# Binning NGVS to CO Nyquist resolution. Not "apples to apples"
# 
%matplotlib widget
plt.close("all")
# 
# u-band data
# 
u_snr = abs(data_u / noise_u)
fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_u)})
img = ax.imshow(u_snr, cmap="plasma", origin="lower", vmin=0, vmax=90) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# CO Nyquist-sampled data
# 
data_co_masked = np.ma.masked_where(data_co <= 0, data_co)
fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_co)})
img = ax.contourf(data_co_masked, levels=15, cmap="magma_r", vmin=0)
# img = ax.imshow(data_co_masked, cmap="magma_r", vmin=0)
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label(r"Surface Density ($\rm M_\odot\; pc^{-2}$)")
ax.set_title("VCC 792 / NGC 4380: Nyquist-sampled CO Data")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# Print stats
# TODO: Clean up this junk!
# 
# ? Don't know if I am passing the arguments in the right order.
# ? Luckily data_co is symmetric: (42, 42)
print("wcs_co", WCS(header_co))
co_bottomleft = WCS(header_co).pixel_to_world(0, 0)
co_topright = WCS(header_co).pixel_to_world(*data_co.shape)
co_centre = WCS(header_co).pixel_to_world(data_co.shape[1] // 2, data_co.shape[0] // 2)
# print(co_centre)
# print(WCS(header_u).world_to_pixel(co_bottomleft))
# print(WCS(header_u).world_to_pixel(co_topright))
# print(WCS(header_u).world_to_pixel(co_centre))
u_centre_of_co = WCS(header_u).world_to_pixel(co_centre)
u_bottomleft_of_co = WCS(header_u).world_to_pixel(co_bottomleft)
u_topright_of_co = WCS(header_u).world_to_pixel(co_topright)
u_to_co_factor = np.subtract(u_topright_of_co, u_bottomleft_of_co)
u_to_co_factor = np.ceil(np.divide(u_to_co_factor, data_co.shape)).astype(int)
u_shape_of_co = u_to_co_factor * data_co.shape
# print(u_to_co_factor)
# print("u_shape_of_co", u_shape_of_co)
# print(WCS(header_u).pixel_to_world(*u_centre_of_co))
# 
# Cut data
# 
data_u_cut, wcs_u_cut = utils.cutout(data_u, co_centre, u_shape_of_co, header=header_u)
noise_u_cut, wcs_noise_u_cut = utils.cutout(noise_u, co_centre, u_shape_of_co, header=header_noise_u)
u_snr_cut = abs(data_u_cut / noise_u_cut)
fig, ax = plt.subplots(subplot_kw={"projection": wcs_u_cut})
img = ax.imshow(u_snr_cut, cmap="plasma", origin="lower", vmin=0, vmax=90) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data Cut")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# Bin to VERTICO resolution
#
data_u_binned, noise_u_binned, x_coords, y_coords, is_good = prelim_bin(
    data_u_cut, noise_u_cut, block_size=u_to_co_factor
)
# * FIXME: Update WCS 
# ? Not sure if slice argument order is correct. Good thing it is symmetric here...
# https://docs.astropy.org/en/stable/api/astropy.wcs.WCS.html#astropy.wcs.WCS.slice
wcs_u_binned = wcs_u_cut.slice(
    (np.s_[::u_to_co_factor[0]], np.s_[::u_to_co_factor[1]])
)
wcs_u_binned.wcs.crpix = np.round(wcs_u_cut.wcs.crpix / u_to_co_factor)
wcs_u_binned.wcs.cd = wcs_u_cut.wcs.cd * u_to_co_factor
print("wcs_u_cut", wcs_u_cut)
print()
print("wcs_u_binned", wcs_u_binned)
# Plot
u_snr_binned = abs(data_u_binned / noise_u_binned)
fig, ax = plt.subplots(subplot_kw={"projection": wcs_u_binned})
# fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_co)})  # not entirely correct
img = ax.imshow(u_snr_binned, cmap="plasma", origin="lower", vmin=150, vmax=1250) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data Binned")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()

In [6]:
# 
# Binning NGVS to CO Nyquist resolution with reproject. "Apples to apples"
# 
%matplotlib widget
plt.close("all")
# 
# Mask the data
# 
data_u_masked = np.ma.masked_where(good_u == 0, data_u)
data_u_masked[data_u_masked.mask] = np.nan  # explicitly set to NaN
noise_u_masked = np.ma.masked_where(good_u == 0, noise_u)
noise_u_masked[noise_u_masked.mask] = np.nan  # explicitly set to NaN
# 
# u-band data
# 
u_snr = abs(data_u_masked / noise_u_masked)
fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_u)})
img = ax.imshow(u_snr, cmap="plasma", origin="lower", vmin=0, vmax=90) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# CO Nyquist-sampled data
# 
data_co_masked = np.ma.masked_where(data_co <= 0, data_co)
fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_co)})
img = ax.contourf(data_co_masked, levels=15, cmap="magma_r", vmin=0)
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='k')
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label(r"Surface Density ($\rm M_\odot\; pc^{-2}$)")
ax.set_title("VCC 792 / NGC 4380: Nyquist-sampled CO Data")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# Print stats
# TODO: Clean up this junk!
# 
# ? Don't know if I am passing the arguments in the right order.
# ? Luckily data_co is symmetric: (42, 42)
# print("wcs_co", wcs_co)
co_bottomleft = wcs_co.pixel_to_world(0, 0)
co_topright = wcs_co.pixel_to_world(*data_co.shape)
co_centre = wcs_co.pixel_to_world(data_co.shape[1] / 2, data_co.shape[0] / 2)
# print("co_centre:", co_centre)
# print(WCS(header_u).world_to_pixel(co_bottomleft))
# print(WCS(header_u).world_to_pixel(co_topright))
# print(WCS(header_u).world_to_pixel(co_centre))
u_centre_of_co = WCS(header_u).world_to_pixel(co_centre)
# print("u_centre_of_co:", u_centre_of_co)
u_bottomleft_of_co = WCS(header_u).world_to_pixel(co_bottomleft)
u_topright_of_co = WCS(header_u).world_to_pixel(co_topright)
u_to_co_factor = np.subtract(u_topright_of_co, u_bottomleft_of_co)
u_to_co_factor_unrounded = np.divide(u_to_co_factor, data_co.shape)
# print("unrounded u_to_co_factor:", u_to_co_factor_unrounded)
u_to_co_factor = np.round(np.divide(u_to_co_factor, data_co.shape)).astype(int)
u_reproject_shape = u_to_co_factor * data_co.shape
# print("u_to_co_factor:", u_to_co_factor)
# print("u_reproject_shape", u_reproject_shape)
# print(WCS(header_u).pixel_to_world(*u_centre_of_co))
# 
# Reproject data
# 
wcs_co_reproject = wcs_co.deepcopy()
wcs_co_reproject.wcs.crpix = wcs_co.wcs.crpix * u_to_co_factor
wcs_co_reproject.wcs.cdelt = wcs_co.wcs.cdelt / u_to_co_factor
wcs_co_reproject.array_shape = u_reproject_shape
# print("wcs_co_reproject", wcs_co_reproject)
u_data_reproj = reproject_interp(
    (data_u_masked, header_u), wcs_co_reproject, shape_out=u_reproject_shape, return_footprint=False
)
u_noise_reproj = reproject_interp(
    (noise_u_masked, header_noise_u), wcs_co_reproject, shape_out=u_reproject_shape, return_footprint=False
)
# print("u_data_reproj.shape", u_data_reproj.shape)
# 
u_snr_reproject = abs(u_data_reproj / u_noise_reproj)
fig, ax = plt.subplots(subplot_kw={"projection": wcs_co_reproject})
img = ax.imshow(u_snr_reproject, cmap="plasma", origin="lower", vmin=0, vmax=90) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of Reprojected u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Reprojected Data")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# Bin to VERTICO resolution
# #
data_u_binned, noise_u_binned, x_coords, y_coords, is_good = prelim_bin(
    u_data_reproj, u_noise_reproj, block_size=u_to_co_factor, func=np.nansum
)
# ? Not sure if slice argument order is correct. Good thing it is symmetric here...
wcs_u_binned = wcs_co_reproject.slice(
    (np.s_[::u_to_co_factor[0]], np.s_[::u_to_co_factor[1]])
)
wcs_u_binned.wcs.crpix = wcs_co_reproject.wcs.crpix / u_to_co_factor
wcs_u_binned.wcs.cdelt = wcs_co_reproject.wcs.cdelt * u_to_co_factor
# print("wcs_u_reproject", wcs_co_reproject)
# print()
# print("wcs_u_binned", wcs_u_binned)
# Plot
u_snr_binned = abs(data_u_binned / noise_u_binned)
fig, ax = plt.subplots(subplot_kw={"projection": wcs_u_binned})
img = ax.imshow(u_snr_binned, cmap="plasma", origin="lower", vmin=150, vmax=1250) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data Binned")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

x_coords, y_coords, signal_binned, noise_binned shapes: (42, 42) (42, 42) (42, 42) (42, 42)
total bad elements (infs/NaNs): 0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close("all")
# Reproject u-band data to CO data
u_data_reproj, u_data_footprint = reproject_exact(
    (data_u, header_u), header_co, parallel=4
)
u_noise_reproj, u_noise_footprint = reproject_exact(
    (noise_u, header_noise_u), header_co, parallel=4
)
u_snr_reproj = abs(u_data_reproj / u_noise_reproj)
# 
# Plot
# 
fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={"projection": WCS(header_co)})
# 
img1 = ax1.imshow(u_snr_reproj, cmap="plasma", origin="lower", vmin=0, vmax=90)
# cbar1 = fig.colorbar(img1)
# cbar1.ax.tick_params(which="both", direction="out")
# cbar1.set_label("SNR of u-Band Data")
# Plot CO contours
ax1.contour(data_co, transform=ax1.get_transform(WCS(header_co)), levels=0, colors='w')
ax1.set_title("u-band Reprojected")
ax1.grid(False)
ax1.set_aspect("equal")
# 
ax2.imshow(u_data_footprint, origin='lower', vmin=0, vmax=1.5)
ax2.set_title("u-band Footprint")
ax2.coords['dec'].set_axislabel_position('r')
ax2.coords['dec'].set_ticklabel_position('r')
ax2.grid(False)
ax2.set_aspect("equal")
fig.suptitle("VCC 792 / NGC 4380")
plt.show()

In [ ]:
plt.close("all")
# 
# Reproject u-band data to CO data
# 
u_data_reproj, u_data_footprint = reproject_interp((data_u, header_u), header_co)
u_noise_reproj, u_noise_footprint = reproject_interp((noise_u, header_noise_u), header_co)
u_snr_reproj = abs(u_data_reproj / u_noise_reproj)
# 
# Plot
# 
fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_co)})
img = ax.imshow(u_snr_reproj, cmap="plasma", origin="lower", vmin=0)
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label(r"SNR of \texttt{reproject\_interp()} u-Band Data")
ax.contour(
    data_co, transform=ax.get_transform(WCS(header_co)), levels=range(0, 15, 5), colors='w'
)
ax.set_title("VCC 792 / NGC 4380: u-band Data Reprojected")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()

In [ ]:
print(u_data_reproj.shape)